# CHAPTER 4 전통 퀀트 투자 전략
--------------------------

* 기술 지표 투자 전략 : 주가 데이터 기반
  - 모멘텀
  - 평균 회귀(mean-reversion)
  > 이동 평균선(moving average)  
  > 상대 강도 지수(relative strength index)  
  > 스토캐스틱 오실레이터(stochastic oscillator)  
  > 등의 지표를 사용하여 구현한다. 
* 가치 투자 전략 : 기업 재무제표 기반
  - 당기순이익
  - 영업이익/영업이익률
  - 매출액
  - 부채비율
  - PER, PBR, PSR, PCR, ROE, ROA
  - 기타 재무제표 외 가치가 있다고 판단되는 부분(EX:CEO의 학벌)
* 마법공식 : 주식시장을 이기는 작은 책, 조엘 그린블라트, 2011
  - 자본 수익률
  - 이익 수익률

## 4.2 평균 회귀 전략
----

* 평균 회귀 : 많은 자료를 토대로 결과를 예측할 때 평균에 가까워지려는 경향
  - 과거부터 특정 종목의 가격이 일정 평균값으로 수렴하는 경향을 관찰
  - 미래 시점에도 일정 값으로 수렴할 것이라고 예측
  - 주가의 평균가격 범위를 이탈할 때 매매


### 4.2.1 볼린저 밴드
-------

* 3개의 선(중심선인 이동 평균선, 상단선과 하단선을 구성하는 표준 편차 밴드)로 구성
> 상단 밴드 = 중간 밴드 + 2 * 20일 이동 표준 편차  
> 중간 밴드 = 20일 이동 평균선  
> 하단 밴드 = 중간 밴드 - 2 * 20일 이동 표준 편차
* 야후 파이낸스에서 SPY 다운로드
  - https://finance.yahoo.com/quote/SPY/history?p=SPY
* 볼린저 밴드 정의

```python
def bollinger_band(df, n, sigma) :
  bb = df.copy()
  # 결측치가 생기므로 쿠션 데이터 확보 필요
  bb['center'] = df['Adj Close'].rolling(window=20).mean()
  bb['ub'] = bb['center'] + sigma * price_df['Adj Close'].rolling(window=20).std()
  bb['lb'] = bb['center'] - sigma * price_df['Adj Close'].rolling(window=20).std()
  return bb
```
* SPY 데이터를 사용하여 볼린저 밴드 만들어보기 

```python
# 파일 업로드 (구글 드라이브 사용)
from google.colab import drive
drive.mount('/content/drive')
# 데이터프레임 만들기
import pandas as pd
fpath = '/content/SPY.csv'
df = pd.read_csv(fpath)
# 볼린저 밴드 생성 시 필요한 변수 (날짜, 수정 종가) 추출
price_df = df.loc[:, ['Date', 'Adj Close']].copy()
price_df.set_index(['Date'], inplace=True)
# 위에서 정의한 함수를 이용해 볼린저 밴드 생성 - 95%
bollinger = bollinger_band(price_df, 20, 2) 
```
* 거래 내역을 기록할 장부 만들기

```python
def create_trade_book(sample) :
  book = sample[['Adj Close']].copy()
  book['trade'] = ''
  return book

base_date = '2009-01-02'
sample = price_df.loc[base_date:]
book = create_trade_book(sample)
```


### 4.2.5 거래 전략
-----

* 전체 코드 : 단기 매매에 효과적

```python
def tradings(sample, book) :
  for i in sample.index :
    # 볼린저 밴드 상한선보다 위에 있을 때 - 매도 시점
    if sample.loc[i, 'Adj Close'] > sample.loc[i, 'ub'] :
      book.loc[i, 'trade'] = ''
    # 볼린저 밴드 하한선보다 아래 있을 때 - 매수 시점
    elif sample.loc[i, 'lb'] > sample.loc[i, 'Adj Close'] :
      if book.shift(1).loc[i, 'trade'] == 'buy' :
        book.loc[i, 'trade'] = 'buy'
      else :
        book.loc[i, 'trade'] = 'buy'
     # 볼린저 밴드 안에 있을 때 - 현재 상태를 유지한다   
    elif sample.loc[i, 'ub'] >= sample.loc[i, 'Adj Close'] and sample.loc[i, 'Adj Close'] >= sample.loc[i, 'lb'] :
      if book.shift(1).loc[i, 'trade'] == 'buy' :
        book.loc[i, 'trade'] = 'buy'
      else :
        book.loc[i, 'trade'] = ''
  return book

book = tradings(sample, book)
```

### 4.2.6 전략 수익률
-------

* 전체 코드

```python
def returns(book) :
  rtn = 1.0
  book['return'] = 1
  buy = 0.0
  sell = 0.0
  for i in book.index :
    if book.loc[i, 'trade'] == 'buy' and book.shift(1).loc[i, 'trade'] == '':
      buy = book.loc[i, 'Adj Close']
      print('진입일 : ', i, ' long 진입가격 : ', buy)
    elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy' :
      sell = book.loc[i, 'Adj Close']
      rtn = (sell - buy) / buy + 1
      book.loc[i, 'return'] = rtn
      print('청산일 : ', i, ' long 진입가격 : ', buy, ' long 청산가격 : ', sell, ' return : ', round(rtn, 4))
    if book.loc[i, 'trade'] == '' :
      buy = 0.0
      sell = 0.0
  acc_rtn = 1.0
  for i in book.index :
    rtn = book.loc[i, 'return']
    acc_rtn = acc_rtn * rtn
    book.loc[i, 'acc return'] = acc_rtn

  print('Accumulated return : ', round(acc_rtn, 4))
  return (round(acc_rtn, 4))
  
returns(book)
```

### 4.2.7 변화 추이
----

* 누적 수익률 그래프로 확인하기

```python
import matplotlib.pylab as plt
book['acc return'].plot()
```

> 평균 회귀 전략이 항상 적중하는 것은 아닌데, 다음과 같은 반례가 있다.  
> - 하락하던 주가가 **파산**해 버린다면 평균으로 회귀 불가
> - 주식을 팔았는데 이후에 **계속해서 오를 수 있음**


## 4.3 듀얼 모멘텀 전략
-----

* 절대 모멘텀 : 과거 대비 현재 시점의 절대적 상승세 평가
  - 최근 N개월간 수익률이 양수이면 매수, 음수이면 공매도
* 상대 모멘텀 : 상대적으로 상승 추세가 강한 종목에 투자
  - 여러개의 종목에 대해 모멘텀이 높은 종목을 매수하고 낮은 종목은 공매도
* 듀얼 모멘텀 : 상대/절대 모멘텀을 결합한 전략


### 4.3.1 듀얼 모멘텀 전략 구현을 위한 절대 모멘텀 전략
-----

* 과거 12개월간 종가의 수익률을 절대 모멘텀 지수로 계산
* 주가 변동이 0% 이상이면 매수 신호, 0% 미만이면 매도 신호 발생 코드 구현
* 데이터 준비하기

```python
import pandas as pd
import numpy as np
import datetime
# 볼린저 밴드 구현 시 다운로드한 SPY.csv 사용
read_df = pd.read_csv(fpath)
# 수정 종가만 가져온다
price_df = read_df.loc[:, ['Date', 'Adj Close']].copy()
price_df['STD_YM'] = price_df['Date'].map(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m'))
month_list = price_df['STD_YM'].unique()
month_last_df = pd.DataFrame()
# 월말 종가 리스트를 만든다
for m in month_list :
  month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == m].index[-1], :])
month_last_df.set_index(['Date'], inplace=True)
```

* 데이터 가공하기

```python
# 1개월 전 종가
month_last_df['BF_1M_Adj Close'] = month_last_df.shift(1)['Adj Close']
# 12개월 전 종가
month_last_df['BF_12M_Adj Close'] = month_last_df.shift(12)['Adj Close']
month_last_df.fillna(0, inplace=True)
```

* 모멘텀 지수 계산 후 포지션 기록할 장부 만들기

```python
book = price_df.copy()
book.set_index(['Date'], inplace=True)
book['trade'] = ''
```

* 거래 실행

```python
ticker = 'SPY'
for x in month_last_df.index :
  signal = ''
  # 절대 모멘텀 계산 : 12개월 전 종가 대비 1개월 전 종가 수익률
  momentum_index = month_last_df.loc[x, 'BF_1M_Adj Close'] / month_last_df.loc[x, 'BF_12M_Adj Close'] - 1
  # 절대 모멘텀 지표 True / False 판단
  # 다른 조건을 추가하고 싶은 경우 other_flag 로 넣는다
  other_flag = True
  flag = True if ((momentum_index > 0.0) and (momentum_index != np.inf) and (momentum_index != -np.inf)) else False and other_flag
  # 절대 모멘텀 지표가 True라면 매수 후 보유(리밸런스 주기)
  if flag :
    signal = 'buy ' + ticker
  print('날짜 : ', x, ' 모멘텀 인덱스 : ', momentum_index, ' flag : ' , flag, ' signal : ', signal)
  book.loc[x:, 'trade'] = signal
```

* 전략 수익률 계산

```python
def returns(book, ticker) :
  rtn = 1.0
  book['return'] = 1
  buy = 0.0
  sell = 0.0
  for i in book.index :
    if book.loc[i, 'trade'] == 'buy ' + ticker and book.shift(1).loc[i, 'trade'] == '' :
      # long 진입
      buy = book.loc[i, 'Adj Close']
      print('진입일 : ', i, ' long 진입가격 : ', buy)
    elif book.loc[i, 'trade'] == 'buy ' + ticker and book.shift(1).loc[i, 'trade'] == 'buy ' + ticker :
      # 보유중
      current = book.loc[i, 'Adj Close']
      rtn = (current - buy) / buy + 1
      book.loc[i, 'return'] = rtn
    elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy ' + ticker :
      # long 청산
      sell = book.loc[i, 'Adj Close']
      rtn = (sell - buy) / buy + 1
      book.loc[i, 'return'] = rtn
      print('청산일 : ', i, ' long 진입가격 : ', buy, ' long 청산가격 : ', sell, ' return : ', round(rtn, 4))
    
    if book.loc[i, 'trade'] == '' :
      buy = 0.0
      sell = 0.0
      current = 0.0

  # 누적 수익률 계산 
  acc_rtn = 1.0
  for i in book.index :
    if book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy ' + ticker :
      rtn = book.loc[i, 'return']
      acc_rtn = acc_rtn * rtn
      book.loc[i:, 'acc return'] = acc_rtn
  print('Accumulated return : ', round(acc_rtn, 4))
  return (round(acc_rtn, 4))

returns(book, ticker)
```

### 4.3.2 듀얼 모멘텀 전략 구현을 위한 상대 모멘텀 전략
-----

* 사전 정의 함수

```python
def data_preprocessing(sample, ticker, base_date):   
  sample['CODE'] = ticker # 종목코드 추가
  sample = sample[sample['Date'] >= base_date][['Date','CODE','Adj Close']].copy() # 기준일자 이후 데이터 사용
  sample.reset_index(inplace= True, drop= True)
  # 기준년월 
  sample['STD_YM'] = sample['Date'].map(lambda x : datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%Y-%m')) 
  sample['1M_RET'] = 0.0 # 수익률 컬럼
  ym_keys = list(sample['STD_YM'].unique()) # 중복 제거한 기준년월 리스트
  return sample, ym_keys

def create_trade_book(sample, sample_codes):
  book = pd.DataFrame()
  book = sample[sample_codes].copy()
  book['STD_YM'] = book.index.map(lambda x : datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%Y-%m'))
  for c in sample_codes:
    book['p '+c] = ''
    book['r '+c] = ''
  return book

# 상대모멘텀 tradings
def tradings(book, s_codes):
  std_ym = ''
  buy_phase = False
  # 종목코드별 순회
  for s in s_codes : 
    print(s)
    # 종목코드 인덱스 순회
    for i in book.index:
      # 해당 종목코드 포지션을 잡아준다. 
      if book.loc[i,'p '+s] == '' and book.shift(1).loc[i,'p '+s] == 'ready ' + s:
        std_ym = book.loc[i,'STD_YM']
        buy_phase = True
      # 해당 종목코드에서 신호가 잡혀있으면 매수상태를 유지한다.
      if book.loc[i,'p '+s] == '' and book.loc[i,'STD_YM'] == std_ym and buy_phase == True : 
        book.loc[i,'p '+s] = 'buy ' + s      
      if book.loc[i,'p '+ s] == '' :
        std_ym = None
        buy_phase = False
  return book

def multi_returns(book, s_codes):
  # 손익 계산
  rtn = 1.0
  buy_dict = {}
  num = len(s_codes)
  sell_dict = {}  
  for i in book.index:
    for s in s_codes:
      if book.loc[i, 'p ' + s] == 'buy '+ s and \
        book.shift(1).loc[i, 'p '+s] == 'ready '+s and \
        book.shift(2).loc[i, 'p '+s] == '' :     # long 진입
          buy_dict[s] = book.loc[i, s]
#         print('진입일 : ',i, '종목코드 : ',s ,' long 진입가격 : ', buy_dict[s])
      elif book.loc[i, 'p '+ s] == '' and book.shift(1).loc[i, 'p '+s] == 'buy '+ s:     # long 청산
        sell_dict[s] = book.loc[i, s]
        # 손익 계산
        rtn = (sell_dict[s] / buy_dict[s]) -1
        book.loc[i, 'r '+s] = rtn
        print('개별 청산일 : ',i,' 종목코드 : ', s , 'long 진입가격 : ', buy_dict[s], ' |  long 청산가격 : ', sell_dict[s],' | return:', round(rtn * 100, 2),'%') # 수익률 계산.
      if book.loc[i, 'p '+ s] == '':     # zero position || long 청산.
        buy_dict[s] = 0.0
        sell_dict[s] = 0.0

  acc_rtn = 1.0        
  for i in book.index:
    rtn  = 0.0
    count = 0
    for s in s_codes:
      if book.loc[i, 'p '+ s] == '' and book.shift(1).loc[i,'p '+ s] == 'buy '+ s: 
        # 청산 수익률 계산.
        count += 1
        rtn += book.loc[i, 'r '+s]
        if (rtn != 0.0) & (count != 0) :
          acc_rtn *= (rtn /count )  + 1
          print('누적 청산일 : ',i,'청산 종목수 : ',count, '청산 수익률 : ',round((rtn /count),4),'누적 수익률 : ' ,round(acc_rtn, 4)) # 수익률 계산.
    book.loc[i,'acc_rtn'] = acc_rtn
  print ('누적 수익률 :', round(acc_rtn, 4))
```

* 과거 1개월 종가의 수익률을 계산하여 상대적으로 높은 수익률을 보인 상위 종목에 매수 신호 발생

```python
import os
import glob
import pandas as pd
import numpy as np
import datetime

# 종목 읽기 - 여러 종목 가져오기
files = glob.glob('../*.csv')

# 필요한 데이터 프레임 생성
# Monthly 데이터를 저장하기 위함이다.
month_last_df = pd.DataFrame(columns=['Date','CODE','1M_RET'])
# 종목 데이터 프레임 생성
stock_df = pd.DataFrame(columns =['Date','CODE','Adj Close'])

for file in files:
  # 데이터 저장 경로에 있는 개별 종목들을 읽어온다.
  if os.path.isdir(file):
    print('%s <DIR> '%file)
  else:
    folder, name = os.path.split(file)
    head, tail = os.path.splitext(name)
    print(file)
    read_df = pd.read_csv(file) # 경로를 읽은 데이터를 하나씩 읽어들인다.
        
    # 1단계. 데이터 가공
    price_df, ym_keys = data_preprocessing(read_df,head,base_date='2010-01-02')
    # 가공한 데이터 붙이기.
    stock_df = stock_df.append(price_df.loc[:,['Date','CODE','Adj Close']],sort=False)
    # 월별 상대모멘텀 계산을 위한 1개월간 수익률 계산
    for ym in ym_keys:
      m_ret = price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],'Adj Close'] / price_df.loc[price_df[price_df['STD_YM'] == ym].index[0],'Adj Close'] 
      price_df.loc[price_df['STD_YM'] == ym, ['1M_RET']] = m_ret
      month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1], ['Date','CODE','1M_RET']])

# 2단계. 상대모멘텀 수익률로 필터링
month_ret_df = month_last_df.pivot('Date','CODE','1M_RET').copy()
# 투자종목 선택할 rank 
month_ret_df = month_ret_df.rank(axis=1, ascending=False, method="max", pct=True) 
# 상위 40%에 드는 종목들만 시그널 목록 만들기
month_ret_df = month_ret_df.where( month_ret_df < 0.4 , np.nan)
month_ret_df.fillna(0,inplace=True)
month_ret_df[month_ret_df != 0] = 1
stock_codes = list(stock_df['CODE'].unique())

#  3단계. signal list로 trading + positioning
sig_dict = dict()
for date in month_ret_df.index:
  # 신호가 포착된 종목코드만 읽어온다.
  ticker_list = list(month_ret_df.loc[date,month_ret_df.loc[date,:] >= 1.0].index)
  # 날짜별 종목코드 저장
  sig_dict[date] = ticker_list
stock_c_matrix = stock_df.pivot('Date','CODE','Adj Close').copy()
book = create_trade_book(stock_c_matrix, list(stock_df['CODE'].unique()))

# positioning
for date,values in sig_dict.items():
    for stock in values:
        book.loc[date,'p '+ stock] = 'ready ' + stock
        
# 3-2  tradings
book = tradings(book, stock_codes)

# 4 단계. 수익률 계산하기.
multi_returns(book, stock_codes)
```

## 4.4 가치 투자 퀀트 전략 (마법공식)
------

* 종목 구성 시 다음과 같이 필터링한다.
  - 거래소에 상장된 개별 종목을 시가총액 순으로 나열한 후 일정 금액 이상 종목 가져오기
  - 이상치(ex: 이익 수익률 지표가 마이너스(-)를 보이는 종목)가 있는 종목 제거
  - 자본 수익률과 이익 수익률을 기준으로 순위를 매겨 투자종목 선별
    * 자본 수익률 : 투입된 자본 대비 수익을 얼마나 올릴 수 있는가  
    * 이익 수익률 : 주가 대비 수익을 얼마나 올릴 수 있는가
> 일반 사용자는 자본 수익률 대신 총자산 순이익률(ROA),   
> 이익 수익률 대신 주가 수익률(PER) 사용  
> ROA가 높은 순, PER이 낮은 순으로 정렬해 저렴하고 우량한 종목 선별

* 마법공식 워크플로우 (158 page 그림 4-8)
  1. 투자에 사용할 자금과 투자 대상 기업 규모 설정
  2. 마법공식에 따른 순위 나열
  3. 자본 수익률 + 이익 수익률 하여 등수가 낮은 순으로 순위매김
  4. 등수가 가장 낮은 5-7개 기업 매수  
  (처음 1년 동안은 투자 금액의 20~30%만 매수)
  5. 나머지 자금을 2~3개월마다 위 과정 반복
  6. 매수 완료된 주식을 1년 동안 보유 후 매도
  7. 매도 후 위 과정 반복

  * 종목 및 재무제표 받아오기

```python
# 네이버 금융, 다음 금융, 야후 파이낸스, FnGuide 에서 받아온 데이터 확인
import pandas as pd
import numpy as np
df = pd.read_csv('./PER_ROA.csv', engine='python')
# 결측치 처리
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
```

  * 순위 계산 실행 함수

```python
def sort_value(s_value, asc = True, standard = 0): 
  '''
  description
      특정 지표값을 정렬한다.
      
  parameters
      s_value : pandas Series
          정렬할 데이터를 받는다.
          
      asc : bool
          True : 오름차순
          False : 내림차순
          
      standard : int
          조건에 맞는 값을 True로 대체하기 위한 기준값 
  returns 
      s_value_mask_rank : pandas Series
          정렬된 순위
  ''' 
  
  s_value_mask = s_value.mask(s_value < standard, np.nan)  #지표별 기준값 미만은 필터링 한다.
  s_value_mask_rank = s_value_mask.rank(ascending=asc, na_option="bottom") # 필터링된 종목에서 순위 선정
  return s_value_mask_rank

per = pd.to_numeric(df['PER'])
roa = pd.to_numeric(df['ROA'])
# per = df['PER']
per_rank = sort_value(per, asc=True, standard=0 ) # PER 지표값을 기준으로 순위 정렬 및 0 미만 값 제거
roa_rank = sort_value(roa, asc=False, standard=0 )# ROA 지표값을 기준으로 순위 정렬 및 0 미만 값 제거

# 최종 순위 생성
result_rank = per_rank + roa_rank   # PER 순위 ROA 순위 합산
result_rank = sort_value(result_rank, asc=True)  # 합산 순위 정렬
result_rank = result_rank.where(result_rank <= 10, 0)  # 합산 순위 필터링
result_rank = result_rank.mask(result_rank > 0, 1)  # 순위 제거

# 순위에 맞는 종목 추출
mf_df = df.loc[result_rank > 0,['종목명','시가총액']].copy() # 선택된 종목 데이터프레임 복사
mf_stock_list = df.loc[result_rank > 0, '종목명'].values # 선택된 종목명 추출

# 종목코드 추가
import FinanceDataReader as fdr
# 코스피, 코스닥, 코넥스의 모든 종목 가져오기
krx_df = fdr.StockListing('KRX')

mf_df['종목코드'] = ''
for stock in mf_stock_list :
    # 데이터프레임에서 조건에 맞는 데이터만 불러온다.
    mf_df.loc[mf_df['종목명'] == stock,'종목코드'] = krx_df[krx_df['Name'] == stock]['Symbol'].values

# 2019년도 마법공식 종목들의 수익률 확인-1
mf_df['2019_수익률'] = ''
for x in mf_df['종목코드'].values : 
    df = fdr.DataReader(x, '2019-01-01','2019-12-31') # 개별 종목 가격 데이터 호출
    cum_ret = df.loc[df.index[-1], 'Close'] / df.loc[df.index[0],'Close'] -1  # 2019년도 누적 수익률 계산
    mf_df.loc[mf_df['종목코드'] == x, '2019_수익률' ] = cum_ret  # 누적 수익률 저장
    df = None

# 수익률 확인-2
mf_df_rtn = pd.DataFrame()
for x in mf_df['종목코드'].values : 
#     print(x ,', ' , mf_df.loc[mf_df['종목코드'] == x, '종목명' ].values[0])
    df = fdr.DataReader(x, '2019-01-01','2019-12-31') # 개별 종목 가격 데이터 호출
    df['daily_rtn'] = df['Close'].pct_change(periods=1)
    df['cum_rtn'] = (1+df['daily_rtn']).cumprod()
    cum_ret = df.loc[df.index[-1],'cum_rtn']
    mf_df.loc[mf_df['종목코드'] == x, '2019_수익률' ] = cum_ret  # 누적 수익률 저장
    df = None
```

> 원래 마법공식 계산법은 EBIT을 활용해 계산된 지표를 사용하지만, 수정된 마법공식에서는 좀 더 간단한 지표를 사용하므로 원래 마법공식과 수익률이 다를 수 있음. 또한, 마법공식을 구성하는 두 가지 지표 중 하나만 사용했을 때 더 높은 수익률을 보일 수도 있음.